In [102]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import xarray
import tifffile
from cellpose import models, io
from skimage import segmentation
import os

In [103]:
path = '/media/gambino/students_workdir/ilias/data/images/tile40/'
cellbound_selection = (1,2,3)

cellbounds = {f'mosaic_Cellbound{cellbound_selection}_z{z_stack}_tile40.tif': np.array([io.imread(path+f'mosaic_Cellbound{i}_z{z_stack}_tile40.tif') for i in cellbound_selection]) for z_stack in range(7)}
dapi = {f'mosaic_DAPI_z{z_stack}_tile40.tif': io.imread(path+f'mosaic_DAPI_z{z_stack}_tile40.tif') for z_stack in range(7)}
avg_intens_cellbounds = {}
for key in cellbounds.keys():
    avg_intens_cellbounds[key] = np.round(np.average(cellbounds[key], axis=0)).astype(np.uint16)

imgs = [np.dstack((dapi[dapi_key], avg_intens_cellbounds[cellbound_key])) for dapi_key, cellbound_key in zip(dapi.keys(), avg_intens_cellbounds.keys())]

Cellpose

<sub>

* channels: 
    * [cytoplasm, nucleus]

    * grayscale = [0,0]

* model_type:
    * 'cyto'
    * 'nuclei'
    * 'cyto2'

In [125]:
#channels
channels = [2,1]

#model eval
model = models.Cellpose(model_type='cyto2', device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
print(f'Using {model.device}')

with torch.no_grad():
    masks, _ , _, diams = model.eval(imgs, batch_size=16, channel_axis=-1, channels=channels, diameter=None)
    torch.cuda.empty_cache()

Using cuda


In [142]:
# save results
dataset = xarray.Dataset(data_vars = dict(images = (['z','row','col','channels'],imgs),
                                        masks = (['z','row','col'], masks_exp)),
                         attrs = dict(dir_path = path, files=[f'({dapi_key},{cellbound_key})' for dapi_key, cellbound_key in zip(dapi.keys(), avg_intens_cellbounds.keys())],
                                       channels = channels, diams=diams))

In [ ]:
out_file=path+'segs/cellpose/mosaic_Cellbound(1,2,3)_DAPI.nc'

dataset.to_netcdf(out_file, engine='netcdf4')